In [ ]:
from pyspark.sql import SparkSession

In [ ]:
import os

os.environ['PYSPARK_SUBMIT_ARGS'] = ' --packages org.apache.spark:spark-sql-kafka-0-10_2.12:3.2.1 pyspark-shell'

In [ ]:
session = (
    SparkSession
    .builder
    .master("local[2]")
    .appName("Driver")
    .getOrCreate()
)
session.sparkContext.setLogLevel('WARN')

In [ ]:
session.stop()

In [ ]:
source = (session.readStream
          .format("kafka")
          .option('kafka.bootstrap.servers', 'localhost:9092')
          .option('subscribe', 'stream_topic')
          ).load()

In [ ]:
df = (source
      .selectExpr('CAST(value AS STRING)', 'offset'))

In [ ]:
df.count()

In [ ]:
console = (df
           .writeStream
           .format('console')
           .outputMode("append"))

In [ ]:
console.start().awaitTermination()

In [ ]:
writer_to_csv = (
    df
    .writeStream
    .format("csv")
    .option("path", "./data")
    .outputMode("append")
)

In [15]:
write_to_kafka = (df
                  .writeStream
                  .format('kafka')
                  .queryName('kafka-output_')
                  .option('kafka.bootstrap.servers', 'localhost:9092')
                  .option('topic', 'output00')
                  .option('checkpointLocation', './.local/checkpoint'))
write_to_kafka.start().awaitTermination()

22/09/08 11:44:42 WARN ResolveWriteToStream: spark.sql.adaptive.enabled is not supported in streaming DataFrames/Datasets and will be disabled.


IllegalArgumentException: Cannot start query with name kafka-output_ as a query with that name is already active in this SparkSession

-------------------------------------------
Batch: 1187
-------------------------------------------


In [11]:
session.conf.set("spark.sql.streaming.checkpointLocation", "./checkPoint")
writer_to_csv.start().awaitTermination()

-------------------------------------------
Batch: 1043
-------------------------------------------
+----------------+------+
|           value|offset|
+----------------+------+
|Data number 1125|  7109|
+----------------+------+

-------------------------------------------
Batch: 1044
-------------------------------------------
+----------------+------+
|           value|offset|
+----------------+------+
|Data number 1126|  7110|
+----------------+------+

-------------------------------------------
Batch: 1045
-------------------------------------------
+----------------+------+
|           value|offset|
+----------------+------+
|Data number 1127|  7111|
+----------------+------+

-------------------------------------------
Batch: 1046
-------------------------------------------
+----------------+------+
|           value|offset|
+----------------+------+
|Data number 1128|  7112|
+----------------+------+

-------------------------------------------
Batch: 1047
--------------------

-------------------------------------------
Batch: 1132
-------------------------------------------
+----------------+------+
|           value|offset|
+----------------+------+
|Data number 1214|  7198|
+----------------+------+

-------------------------------------------
Batch: 1133
-------------------------------------------
+----------------+------+
|           value|offset|
+----------------+------+
|Data number 1215|  7199|
+----------------+------+

-------------------------------------------
Batch: 1134
-------------------------------------------
+----------------+------+
|           value|offset|
+----------------+------+
|Data number 1216|  7200|
+----------------+------+

-------------------------------------------
Batch: 1135
-------------------------------------------
+----------------+------+
|           value|offset|
+----------------+------+
|Data number 1217|  7201|
+----------------+------+

-------------------------------------------
Batch: 1136
--------------------

ERROR:root:KeyboardInterrupt while sending command.
Traceback (most recent call last):
  File "/home/dmitryrusack/PycharmProjects/SparkArticle/venv/lib/python3.8/site-packages/py4j/java_gateway.py", line 1038, in send_command
    response = connection.send_command(command)
  File "/home/dmitryrusack/PycharmProjects/SparkArticle/venv/lib/python3.8/site-packages/py4j/clientserver.py", line 511, in send_command
    answer = smart_decode(self.stream.readline()[:-1])
  File "/usr/lib/python3.8/socket.py", line 669, in readinto
    return self._sock.recv_into(b)
KeyboardInterrupt


KeyboardInterrupt: 

In [ ]:
accum = session.sparkContext.accumulator(0)
foreach_data = (
    df
    .writeStream
    .foreach(lambda x: accum.add(1))
    .start()
    .awaitTermination()
)

In [ ]:
accum.value

In [16]:
session.stop()

22/09/08 11:45:15 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@e5b1dda is aborting.
22/09/08 11:45:15 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@e5b1dda aborted.
22/09/08 11:45:15 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@118f92f7 is aborting.
22/09/08 11:45:15 ERROR WriteToDataSourceV2Exec: Data source write support org.apache.spark.sql.execution.streaming.sources.MicroBatchWrite@118f92f7 aborted.
22/09/08 11:45:15 ERROR FileFormatWriter: Aborting job ba159fd9-71d0-402b-acb7-63bd7c636171.
java.lang.IllegalStateException: SparkContext has been shutdown
	at org.apache.spark.SparkContext.runJob(SparkContext.scala:2220)
	at org.apache.spark.sql.execution.datasources.FileFormatWriter$.write(FileFormatWriter.scala:245)
	at org.apache.spark.sql.execution.strea